# Import Libraries

In [1]:
import pandas as pd
from itertools import zip_longest
from bs4 import BeautifulSoup
from selenium import webdriver

from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time

C:\Users\Gilbert KH\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# code to force show all column
pd.set_option('display.max_columns', None)

# max column width
pd.set_option('display.max_colwidth', 300)

# Load Dataset

In [3]:
df = pd.read_csv('../datasets/rumah123_hyperlink.csv')

# Scrap Web Per Pages

In [4]:
#init webdriver & test BeautifulSoup
driver = webdriver.Chrome()

url="https://www.rumah123.com/jual/rumah/?q=Balikpapan"
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html, "html.parser") #Menentukan parser HTML yang ingin kita gunakan.
print(soup.prettify()[:700]) # memberikan representasi visual dari pohon parse yang dibuat dari konten HTML mentah.

<html lang="id-ID">
 <head>
  <script async="" id="analytics-js" src="https://cdn.segment.com/analytics.js/v1/fpueXeNRJ8uBgCLuBDRDkZFAMBT0yLLb/analytics.min.js" type="text/javascript">
  </script>
  <script>
   partytown = {
                "debug": false,
                "lib": "/asset-core/script/partytown/",
                "logCalls": true,
                "logGetters": true,
                "logImageRequests": true,
                "logMainAccess": true,
                "logScriptExecution": true,
                "logSendBeaconRequests": true,
                "logStackTraces": true,
                "forward": ['dataLayer.push', 'fbq', '_hsq.push', 'ttq.track', 'ttq.page', 'ttq.load'],
 


### Changed Logic
1. First initial labels of all possible data
2. Create a dictionary based on the labels
3. Create a Loop to scrap the data and store it in the dictionary
    - First, fill the dictionary with NaN for all key so that when we didn't found the data, it still in the same shape
    - Loop 1: To get the labels1 data (Kamar Tidur, Kamar Mandi, etc)
    - Loop 2: To get the labels2 data (Luas Bangunan, Luas Tanah, etc)

In [5]:
# clean dictionary
data1 = {}
data2 = {}
harga = []
judul = []
lokasi = []
urls = []
img_hyperlinks = []
harga = [np.nan]*1467
judul = [np.nan]*1467
lokasi = [np.nan]*1467
urls = [np.nan]*1467
img_hyperlinks = [np.nan]*1467

# define the labels
labels1 = ['Kamar Tidur', 'Kamar Mandi', 'Luas Tanah', 'Luas Bangunan', 'Carport', 'Tipe Properti', 'Sertifikat', 'Daya Listrik']
labels2 = ['Kamar Pembantu', 'Dapur', 'Ruang Makan', 'Ruang Tamu', 'Kondisi Perabotan', 'Jumlah Lantai', 'Pemandangan', 'Terjangkau Internet', 'Lebar Jalan', 'Sumber Air', 'Hook', 'Kondisi Properti', 'Tipe Iklan']

# create dictionaries to store the labels and their values
data1 = {label: [np.nan]*1467 for label in labels1}
data2 = {label: [np.nan]*1467 for label in labels2}

loop = 776
print("Progresss: ")

# traverse the hyperlink to get the data needed (test 10 data first)
for i in df['hyperlink'][776:1000]:
    
    url = i
    urls[loop] = url
    img_hyperlinks[loop] = df['img_hyperlinks'][loop]
    
    url=i
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # generate random number between 1 and 3
    rand = np.random.randint(1, 17)

    # short pause if loop is a multiple of 5, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 5) == 0) and (loop != 0):
        pause = 25 - rand
        time.sleep(pause)
        # print(f'Long Pause for {pause} seconds...')
    
    # Long pause if loop is a multiple of 19, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 19) == 0) and (loop != 0):
        pause = 79 - rand
        print(f'Long Pause for {pause} seconds...')
        time.sleep(pause)
    
    # Super long pause if loop is a multiple of 5, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 98) == 0) and (loop != 0):
        pause = 820 - rand
        print(f'Super Long Pause for {pause} seconds...')
        time.sleep(pause)
    
    
    get_price_iklan = soup.find_all('div', class_="r123-listing-summary__price")
    # get the span inside the div
    for i in get_price_iklan:
        get_price_iklan = i.find('span')  # use find instead of find_all
        harga[loop] = get_price_iklan.text

    judul_elements = soup.find_all('h1', {"class":"r123-listing-summary__header-container-title"})
    if judul_elements:
        judul[loop] = judul_elements[0].get_text()

    lokasi_elements = soup.find_all('div', {"class":"r123-listing-summary__header-container-address"})
    if lokasi_elements:
        lokasi[loop] = lokasi_elements[0].get_text()
    
    # ----------------- #

    # first loop
    spec_items = soup.find_all('div', class_="listing-specification-v2__item")
    for item in spec_items:
        label = item.find('span', class_="listing-specification-v2__item-label").text
        value = item.find('span', class_="listing-specification-v2__item-value").text

        # if the label is in the first dictionary, insert the value at the index of the current loop
        if label in data1:
            data1[label][loop] = value

    # second loop
    interior_items = soup.find_all('div', class_="ui-molecule-accordion-r123")
    for item in interior_items:
        spec_items = item.find_all('div', class_="listing-specification-v2__item")
        for spec_item in spec_items:
            label = spec_item.find('span', class_="listing-specification-v2__item-label").text
            value = spec_item.find('span', class_="listing-specification-v2__item-value").text

            # if the label is in the second dictionary, insert the value at the index of the current loop
            if label in data2:
                data2[label][loop] = value
   
    #----------------- #
    loop += 1
    
    # print loop progress total number of link divided by hyperlinks
    print(f'Done: {loop} / {df["hyperlink"].size}')    

Progresss: 
Done: 777 / 1467
Done: 778 / 1467
Done: 779 / 1467
Long Pause for 75 seconds...
Done: 780 / 1467
Done: 781 / 1467
Done: 782 / 1467
Done: 783 / 1467
Done: 784 / 1467
Super Long Pause for 811 seconds...
Done: 785 / 1467
Done: 786 / 1467
Done: 787 / 1467
Done: 788 / 1467
Done: 789 / 1467
Done: 790 / 1467
Done: 791 / 1467
Done: 792 / 1467
Done: 793 / 1467
Done: 794 / 1467
Done: 795 / 1467
Done: 796 / 1467
Done: 797 / 1467
Done: 798 / 1467
Long Pause for 78 seconds...
Done: 799 / 1467
Done: 800 / 1467
Done: 801 / 1467
Done: 802 / 1467
Done: 803 / 1467
Done: 804 / 1467
Done: 805 / 1467
Done: 806 / 1467
Done: 807 / 1467
Done: 808 / 1467
Done: 809 / 1467
Done: 810 / 1467
Done: 811 / 1467
Done: 812 / 1467
Done: 813 / 1467
Done: 814 / 1467
Done: 815 / 1467
Done: 816 / 1467
Done: 817 / 1467
Long Pause for 71 seconds...
Done: 818 / 1467
Done: 819 / 1467
Done: 820 / 1467
Done: 821 / 1467
Done: 822 / 1467
Done: 823 / 1467
Done: 824 / 1467
Done: 825 / 1467
Done: 826 / 1467
Done: 827 / 146

In [6]:
# convert the dictionaries to dataframes
judull = pd.DataFrame(judul, columns=['Judul'])
hargaa = pd.DataFrame(harga, columns=['Harga'])
lokasia = pd.DataFrame(lokasi, columns=['Lokasi'])
hyperlink_df = pd.DataFrame(urls, columns=['Hyperlink'])
img_hyperlink_df = pd.DataFrame(img_hyperlinks, columns=['Img_Hyperlink'])

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# concatenate the dataframes
df_fix = pd.concat([judull, hargaa, lokasia, hyperlink_df, img_hyperlink_df, df1, df2], axis=1)
print(df_fix.shape)

# drop all rows where Judul is NaN
df_fix = df_fix.dropna(subset=['Judul'])

df_fix

(1467, 26)


,Judul,Harga,Lokasi,Hyperlink,Img_Hyperlink,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
776,RUMAH DIJUAL BALIKPAPAN BARU VANCOUVER HV NO.15 BALIKPAPAN,"Rp 4,4 Miliar","Balikpapan Baru, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos16069690/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1727362/2024-01-16-15-53-39-06d4dd14-6782-498a-9582-a134c0121f83.jpg,4,4,342 m²,102 m²,2,Rumah,SHM - Sertifikat Hak Milik,5500 Watt,1,2,Ya,Ya,Semi Furnished,2,NaN,Tidak,NaN,Sumur Bor,Tidak,Sudah Renovasi,Dijual
777,"RUMAH DI BALIKPAPAN, BISA BUAT TEMPAT USAHA HARGA DI BWH PASAR","Rp 2,85 Miliar","Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos16058571/,https://picture.rumah123.com/r123/720x420-crop/house/ho16/16058571/original/hos16058571-rumah-di-jual-di-balikpapan-utara-balikpapan-1705372328.jpg,5,3,401 m²,400 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,4400 Watt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
778,"RUMAH JUAL PUPUK,DEKAT BSB DAN BANDARA","Rp 4,6 Miliar","Balikpapan Kota, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos15992326/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1929388/2024-01-09-07-32-52-d2bb9259-b469-4e25-a377-c0726bdcb9a2.jpg,4,2,330 m²,300 m²,2,Rumah,SHM - Sertifikat Hak Milik,10000 Watt,1,1,Ya,Ya,Furnished,1,Perkotaan,Ya,2 Mobil,PAM atau PDAM,Tidak,Sudah Renovasi,Dijual
779,MURAH RUMAH BALIKPAPAN REGENCY 3 LANTAI,"Rp 1,24 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos15989815/,https://picture.rumah123.com/r123/720x420-crop/house/ho15/15989815/original/hos15989815-rumah-di-jual-di-balikpapan-selatan-balikpapan-1704777388.jpg,4,2,120 m²,145 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
780,RUMAH SIAP HUNI 2 LANTAI SEMI FURNISH DENGAN DESAIN MINIMALIS LOKASI AMAN DAN NYAMAN,"Rp 1,2 Miliar","Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos15985985/,https://picture.rumah123.com/r123/720x420-crop/house/ho15/15985985/original/hos15985985-rumah-di-jual-di-balikpapan-utara-balikpapan-1704760039.jpg,2,2,78 m²,47 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,RUMAH + TANAH LUAS PINGGIR JALAN RAYA,Rp 12 Miliar,"Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos10286837/,https://picture.rumah123.com/r123/720x420-crop/house/ho10/10286837/original/hos10286837-rumah-di-jual-di-balikpapan-utara-balikpapan-1653373187.jpg,6,5,1270 m²,500 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
996,RUMAH PRAPATAN,Rp 430 Juta,"Prapatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos10286828/,https://picture.rumah123.com/r123/720x420-crop/house/ho10/10286828/original/hos10286828-rumah-di-jual-di-prapatan-balikpapan-1653373152.jpg,3,1,180 m²,60 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
997,RUMAH MINIMALIS WIKA MAHOGANY,Rp 700 Juta,"Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos10286821/,https://picture.rumah123.com/r123/720x420-crop/house/ho10/10286821/original/hos10286821-rumah-di-jual-di-balikpapan-utara-balikpapan-1653373120.jpg,2,2,120 m²,120 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
998,RUMAH MINIMALIS DIKAWASAN WIKA,Rp 750 Juta,"Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos10242392/,https://picture.rumah123.com/r123/720x420-crop/house/ho

In [7]:
# check null values
df_fix.isnull().sum()

Judul                    0
Harga                    0
Lokasi                   0
Hyperlink                0
Img_Hyperlink            0
Kamar Tidur              4
Kamar Mandi              4
Luas Tanah               0
Luas Bangunan            0
Carport                190
Tipe Properti            0
Sertifikat               0
Daya Listrik             9
Kamar Pembantu         189
Dapur                  184
Ruang Makan            113
Ruang Tamu             113
Kondisi Perabotan      174
Jumlah Lantai          170
Pemandangan            177
Terjangkau Internet    113
Lebar Jalan            176
Sumber Air             177
Hook                   113
Kondisi Properti       155
Tipe Iklan               0
dtype: int64

In [8]:
# save to csv
df_fix.to_csv('../datasets/rumah123_data_single_raw_fix_part_2_b.csv')

In [9]:
# print only null in kamar
df_fix[df_fix['Kamar Tidur'].isnull()]

,Judul,Harga,Lokasi,Hyperlink,Img_Hyperlink,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
798,rumah megah dan cantik di balikpapan baru\n,Rp 5 Miliar,"Balikpapan Baru, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos15613033/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1730302/2023-11-28-10-39-09-f803d40b-d487-47f3-8bb9-ad666e9e026f.jpg,NaN,NaN,300 m²,400 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,Tidak,Tidak,NaN,1,NaN,Tidak,NaN,PAM atau PDAM,Tidak,Bagus,Dijual
864,Di Jual Rumah di gn.Belah.Balikpapan,Rp 600 Juta,"Balikpapan Tengah, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos14143557/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1729095/2023-07-04-05-09-26-4ed20397-4e17-4558-b00d-5a14e1eef275.jpg,NaN,NaN,106 m²,42 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,900 Watt,NaN,NaN,Tidak,Tidak,NaN,1,NaN,Tidak,NaN,PAM atau PDAM,Tidak,Bagus,Dijual
865,Rumah Tinggal dan Rumah Kost jalan Mt.Haryono,Rp 3 Miliar,"Balikpapan Kota, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos14143446/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1729095/2023-07-04-04-58-12-29bbb3fc-3e15-4396-b1f3-de29d4efce68.jpg,NaN,NaN,378 m²,500 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,Tidak,Tidak,NaN,1,NaN,Tidak,NaN,PAM atau PDAM,Tidak,Sudah Renovasi,Dijual
945,Rumah di jalan Mulawarman Lamaru,Rp 400 Juta,"Balikpapan Timur, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos13247547/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1729095/2023-03-28-04-07-57-4e19518f-1132-43df-8ebc-2485200f8db8.jpg,NaN,NaN,112 m²,112 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,1,Tidak,Tidak,NaN,1,NaN,Tidak,NaN,Sumur Bor,Tidak,Baru,Dijual


In [12]:
# # concat raw fix part 2 a and b
# df_fix_part_2_a = pd.read_csv('../datasets/rumah123_data_single_raw_fix_part_2_a.csv')
# df_fix_part_2_b = pd.read_csv('../datasets/rumah123_data_single_raw_fix_part_2_b.csv')

# df_fix_part_2 = pd.concat([df_fix_part_2_a, df_fix_part_2_b], axis=0)

# # to csv
# df_fix_part_2.to_csv('../datasets/rumah123_data_single_raw_concated_part_2.csv')

In [11]:
# df_fix_part_2

,Unnamed: 0,Judul,Harga,Lokasi,Hyperlink,Img_Hyperlink,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
0,501,RUMAH LUAS MEWAH DENGAN KOLAM RENANG DALAM KAWASAN RUMAH ELITE BALIKPAPAN,"Rp 9,5 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos12030548/,https://picture.rumah123.com/r123/720x420-crop/house/ho12/12030548/original/hos12030548-rumah-di-jual-di-balikpapan-selatan-balikpapan-1670476856.jpg,4.0,3.0,670 m²,400 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,13200 Watt,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1,502,Rumah Jalan Siaga Balikpapan.,"Rp 2,8 Miliar","Balikpapan Kota, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos11737208/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1729095/2022-11-10-04-26-39-870666c1-84da-4fbf-bccb-e83de21efd00.jpg,NaN,NaN,320 m²,110 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3300 Watt,NaN,1.0,Tidak,Tidak,NaN,1.0,NaN,Tidak,NaN,PAM atau PDAM,Tidak,Bagus,Dijual
2,503,Rumah ellite full furnish di tengah kota,"Rp 1,8 Miliar","Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos11498605/,https://picture.rumah123.com/r123/720x420-crop/house/ho11/11498605/original/hos11498605-rumah-di-jual-di-balikpapan-utara-balikpapan-1666253473.jpg,4.0,4.0,186 m²,160 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,Bagus,Dijual
3,504,RUMAH BDI,"Rp 1,7 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos11494154/,https://picture.rumah123.com/r123/720x420-crop/house/ho11/11494154/original/hos11494154-rumah-di-jual-di-balikpapan-selatan-balikpapan-1666234734.jpg,4.0,3.0,180 m²,100 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
4,505,RUMAH LUAS MEWAH DEPAN TAMAN DAN LAPANGAN BASKET DALAM PERUMAHAN ELITE BALIKPAPAN,"Rp 9,5 Miliar","Balikpapan Baru, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos11417461/,https://picture.rumah123.com/r123/720x420-crop/house/ho11/11417461/original/hos11417461-rumah-di-jual-di-balikpapan-baru-balikpapan-1665628960.jpg,4.0,4.0,396 m²,289 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,5500 Watt,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,995,RUMAH + TANAH LUAS PINGGIR JALAN RAYA,Rp 12 Miliar,"Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos10286837/,https://picture.rumah123.com/r123/720x420-crop/house/ho10/10286837/original/hos10286837-rumah-di-jual-di-balikpapan-utara-balikpapan-1653373187.jpg,6.0,5.0,1270 m²,500 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
219,996,RUMAH PRAPATAN,Rp 430 Juta,"Prapatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos10286828/,https://picture.rumah123.com/r123/720x420-crop/house/ho10/10286828/original/hos10286828-rumah-di-jual-di-prapatan-balikpapan-1653373152.jpg,3.0,1.0,180 m²,60 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
220,997,RUMAH MINIMALIS WIKA MAHOGANY,Rp 700 Juta,"Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos10286821/,https://picture.rumah123.com/r123/720x420-crop/house/ho10/10286821/original/hos10286821-rumah-di-jual-di-balikpapan-utara-balikpapan-1653373120.jpg,2.0,2.0,120 m²,120 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
221,998,RUMAH MINIMALIS DIKAWASAN WIKA,Rp 750 Juta,"Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos10242392/,https://picture.rumah123.com/r123/720x420-